# Advent of Code 2024 Day 13 

### Setup

In [97]:
from aocd import get_data, submit

day = 13
year = 2024


In [ ]:
with open('example.txt', 'r') as file:
    raw_sample_data = "".join(file.readlines())

raw_sample_data[:100]

In [ ]:
raw_test_data = get_data(day=day, year=year)

raw_test_data[:]

##### Data Parsing

In [100]:
def trim_label(line:str, delimiter:str = ':'):
    return line.split(delimiter)[1].strip()

In [101]:
def get_coords(line:str, delim:str = ','):
    """Extracts the x and y coordinates from a string
    
    Example: 'X+94, Y+34' -> (94, 34)
    """
    coords_raw = [ c.strip() for c in line.split(delim) ]
    x, y = int(coords_raw[0][2:]), int(coords_raw[1][2:])

    return (x, y)

In [ ]:
def parse_data(raw_data:str):
    data = [{}]
    
    data_idx = 0
    for idx, line in enumerate(raw_data.split('\n')):
        if line == '':
            continue
        
        if 'Button A' in line:
            data[data_idx]['A'] = get_coords(trim_label(line))
            continue

        elif 'Button B' in line:
            data[data_idx]['B'] = get_coords(trim_label(line))
            continue

        elif 'Prize' in line:
            data[data_idx]['PrizeLocation'] = get_coords(trim_label(line))
            data_idx += 1
            if idx + 1 < len(raw_data.split('\n')):
                data.append({})
            continue

    return data


sample_data = parse_data(raw_sample_data)
test_data = parse_data(raw_test_data)

sample_data

### Part One!

In [103]:
use_sample_data = False
part = 'a'

In [ ]:
data = sample_data if use_sample_data else test_data

data

In [ ]:
max_iters = 100
a_cost, b_cost = 3, 1

solutions = []
for config in data:
    min_tokens = None
    for a in range(max_iters):
        for b in range(max_iters):
            tokens = (a * a_cost) + (b * b_cost)
            if min_tokens is not None and tokens > min_tokens:
                continue

            prize = config['PrizeLocation']
            x = a * config['A'][0] + b * config['B'][0]
            y = a * config['A'][1] + b * config['B'][1]
            
            if (x, y) == prize:
                min_tokens = tokens if min_tokens is None or tokens < min_tokens else min_tokens
    
    if min_tokens is not None:
        solutions.append(min_tokens)

sum(solutions)

In [ ]:

part_a_answer = sum(solutions)
part_a_answer

In [ ]:
if not use_sample_data and part == 'a':
    submit(answer=part_a_answer, part='a', day=day, year=year, reopen=True)

### Part Two!

In [115]:
use_sample_data = False
part='b'

In [ ]:
def parse_data_part02(raw_data:str):
    data = [{}]
    boost = 10000000000000
    
    data_idx = 0
    for idx, line in enumerate(raw_data.split('\n')):
        if line == '':
            continue
        
        if 'Button A' in line:
            data[data_idx]['A'] = get_coords(trim_label(line))
            continue

        elif 'Button B' in line:
            data[data_idx]['B'] = get_coords(trim_label(line))
            continue

        elif 'Prize' in line:
            x, y = get_coords(trim_label(line))
            coords = (x + boost, y + boost)
            data[data_idx]['PrizeLocation'] = coords
            data_idx += 1
            if idx + 1 < len(raw_data.split('\n')):
                data.append({})
            continue

    return data


sample_data = parse_data_part02(raw_sample_data)
test_data = parse_data_part02(raw_test_data)

sample_data

In [ ]:
data = sample_data if use_sample_data else test_data
data[:10]

In [ ]:
x = data[0]
x

In [ ]:
from scipy.optimize import linprog
import numpy as np

a_cost, b_cost = 3, 1

solutions = []
for config in data:
    cost_fn = np.array([a_cost, b_cost])
    a_eq = np.array([config['A'], config['B']]).T
    b_eq = np.array(config['PrizeLocation'])

    result = linprog(cost_fn, A_eq=a_eq, b_eq=b_eq, method="highs")

    if result.success:
        # need to see if the float solution matches the int solution
        a, b = np.round(result.x).astype(int)

        x = a * config['A'][0] + b * config['B'][0]
        y = a * config['A'][1] + b * config['B'][1]

        if (x, y) == tuple(config['PrizeLocation']):
            tokens = a * a_cost + b * b_cost
            solutions.append(tokens)

sum(solutions)

In [ ]:
part_b_answer = sum(solutions)
part_b_answer

In [ ]:
if not use_sample_data and part == 'b':
    submit(answer=part_b_answer, part='b', day=day, year=year, reopen=True)